In [42]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

In [62]:
def fetch_data(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    return soup

def find_links(soup):
    links = [a['href'] for a in soup.find_all('a', href=True, class_='title')]
    
    return links

def find_info(soup):
    job_titles = soup.find_all('div', id='contenttitle')
    job_titles = [title.text.strip() for title in job_titles]
    descriptions = soup.find_all('div', class_='desc')
    descriptions = [desc.text.strip() for desc in descriptions]
    phone_numbers = soup.find_all('span', class_='mobilestyle')
    phone_numbers = [phone.text.strip() for phone in phone_numbers]
    locations = soup.find_all('div', class_='viewdesc')
    locations = [location.text.replace("位置：", "").strip() for location in locations if "位置：" in location.text]
    times = soup.find_all('div', class_='posttime')
    times = [time.text.split('：')[-1].strip() for time in times]
    return job_titles, descriptions,locations,times,phone_numbers

web_url = "https://www.168worker.com/list/1_0/"
pages = 2  # 抓取的页数

In [63]:
with open('results.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(['标题', '描述', '位置', '时间', '电话'])

for i in range(1, pages + 1):
    page_url = web_url + str(i)
    data = fetch_data(page_url)
    links = find_links(data)
    for link in links:
        link = requests.compat.urljoin(page_url, link)
        data = fetch_data(link)
        job_titles, descriptions,locations,times,phone_numbers = find_info(data)
        with open('results.csv', 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            for title, desc, location, time, number in zip(job_titles, descriptions, locations, times, phone_numbers):
                writer.writerow([title, desc, location, time, number])

In [64]:
df = pd.read_csv('results.csv')
df

,标题,描述,位置,时间,电话
0,"麻省中日餐急需熟手三手寿司师傅, Quincy有接送","麻省中日餐诚急需熟手三手寿司师傅， 工作环境優, Quincy 接送，有意者请电联或发信息 ...",麻州/波士顿,2023/07/26,(617) 763-2156
1,诚请经验企台,堂吃店诚请经验企台，单间，周一全休，离波士顿两个小时，每天都𠕇往返车辆，意者电9789566...,新罕布什尔,2023/07/20,(978) 956-6688
2,曼哈顿中餐厅诚聘有经验企台/收银,曼哈顿中餐厅请企台，以及 收银兼打包，底薪小费均优\n要求：\n1，有企台经验，学习能力强，...,纽约/曼哈顿,2023/07/20,(929) 307-7777
3,央厨厨师/预制菜(工厂)品研发经理,支持工作签证和绿卡！！不要打电话！！\n发送简历至邮箱：[email protected]\...,纽约/法拉盛,2023/06/22,(929) 500-5986
4,餐馆洗碗清洁兼打什,位于奥兰多东面的Melbourne市，诚聘洗碗、 清潔、兼打什一名， 需有合法工作证。只要勤...,佛州/奥兰多,2023/06/04,(321) 960-8933
...,...,...,...,...,...
195,新泽西中餐诚请下周一兼职收银，有意者请联系：9173854289,工作轻松，法拉盛接送,新泽西,2023/07/21,(917) 385-4289
196,法拉盛union街招聘会帮炒的后厨,法拉盛union街招聘会帮炒的后厨 有意者请联系9173197577,纽约/法拉盛,2023/07/21,(917) 319-7577
197,皇后区Astoria小吃店招熟手企台,熟手企台，11:30-9:45 休息一小时 小费好，包低金 9173328800,纽约/皇后区,2023/07/21,(917) 332-8800
198,中日餐堂吃店高薪诚聘炒锅,佛州棕榈海岸附近中日餐堂吃店诚聘炒锅，薪优，住宿条件好（网络，单间）有意电646591760...,佛州/奥兰多,2023/07/20,(646) 591-7607
